In [5]:
import numpy as np
from scipy.integrate import quad


In [ ]:
def ptmts(ts, tm, sigma):
    '''
    Computes the likelihood of observing a measurement `tm` given the true stimulus duration `ts`
    and the standard deviation `sigma` of the measurement noise (assumed Gaussian).

    Args:
        ts (array-like): True durations of the stimulus.
        tm (array-like): Measured durations of the stimulus.
        sigma (float): Standard deviation of the likelihood

    Returns:
        np.ndarray: Likelihood values, as a NumPy array (column vector if inputs are column vectors).
    '''
    return np.exp((-(tm - ts)**2) / (2 * sigma**2)) / (np.sqrt(2 * np.pi) * sigma)


In [17]:
f = lambda x : np.sin(x)
result, error = quad(f,0,np.pi)

In [18]:
error

2.220446049250313e-14

In [ ]:

def BLS_est(ts,tm,sigma):
    '''
    
    Computes Bayesian Least Square estimates

    Args:
        ts (array-like): True durations of the stimulus.
        tm (array-like): Measured durations of the stimulus.
        sigma (float): Standard deviation of the likelihood

    Returns:
        np.ndarray:  te: (array-like): estimate of duration of stimulus
    '''

    return quad(lambda ts,tm,sigma:ts*ptmts(ts,tm,sigma),min(ts),max(ts))/quad(lambda ts,tm,sigma:ptmts(ts,tm,sigma),min(ts),max(ts))

In [21]:

def inv_BLS_est(ts,tm0,te,sigma):
    '''
    Finds the tm that generates the te closest to the observed value.
    It does so by empirically inverting the BLS function.

    Args:
        ts (np.ndarray): Column vector (N,) — true stimulus durations
        tm0 (np.ndarray): Column vector (M,) — candidate measured durations
        te (np.ndarray): Column vector (N,) — observed BLS estimates
        sigma (float): Standard deviation of likelihood

    Returns:
        np.ndarray: Column vector (N,) of tm values estimated to generate each te
    '''
    te_hat_matrix = BLS_est(ts, tm0, sigma)  # shape (N, M)

    error_matrix = np.abs(te_hat_matrix - te[:, np.newaxis])  # shape (N, M)

    best_indices = np.argmin(error_matrix, axis=1)  # shape (N,)

    tm_est = tm0[best_indices]  # shape (N,)

    return  tm_est.reshape(-1, 1) # column vector


In [22]:
def NeglogLike(ts,tm0,te,sigma):
    '''
    Compute negative log likelihood

    Args:
    ts (np.ndarray): Column vector (N,) — true stimulus durations
    tm0 (np.ndarray): Column vector (M,) — candidate measured durations
    te (np.ndarray): Column vector (N,) — observed BLS estimates
    sigma (float): Standard deviation of likelihood

    Returns:
    Negative Log Likelihood : scalar , float
    '''

    tm = inv_BLS_est(ts, tm0, te, sigma)
    p = ptmts(ts,tm,sigma)
    return -np.sum(np.log(p))